In [10]:
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

# Definir la estructura de la red bayesiana
modelo = BayesianNetwork([('Nublado', 'Lluvia'),
                          ('Nublado', 'Aspersor'),
                          ('Lluvia','HierbaMojada'),
                          ('Aspersor','HierbaMojada')])

# Definir las probabilidades condicionales (cpd)
cpd_nublado = TabularCPD(variable='Nublado', variable_card=2, values=[[0.5], [0.5]], state_names={'Nublado': [False, True]})

cpd_lluvia = TabularCPD(variable='Lluvia', variable_card=2,
                        values=[[0.80, 0.20],
                                [0.20, 0.80]],
                        evidence=['Nublado'],
                        evidence_card=[2],
                        state_names={
                                'Lluvia': [False, True],
                                'Nublado': [False, True]
                        })

cpd_aspersor = TabularCPD(variable='Aspersor', variable_card=2,
                        values=[[0.50,0.90],
                                [0.50,0.10]],
                        evidence=['Nublado'],
                        evidence_card=[2],
                        state_names={
                                'Aspersor': [False, True],
                                'Nublado': [False, True]
                        })

cpd_hierbaMojada = TabularCPD(variable='HierbaMojada', variable_card=2,
                        values=[[1,0.10,0.10,0.01],
                        [0,0.90,0.90,0.99]],
                        evidence=['Aspersor','Lluvia'],
                        evidence_card=[2,2],
                        state_names={
                                'HierbaMojada': [False, True],
                                'Aspersor': [False, True],
                                'Lluvia': [False, True]
                        })

#Añadir los CPD al modelo.
modelo.add_cpds(cpd_nublado, cpd_lluvia, cpd_aspersor, cpd_hierbaMojada)

#Comprobar que el modelo es válido.
assert modelo.check_model()

# Realizar inferencia usando Variable Elimination
inferencia = VariableElimination(modelo)

# Calcular P(Lluvia = Cierto | HierbaMojada = cierto)
query_result = inferencia.query(variables=['Lluvia'], evidence={'HierbaMojada': True})
print(query_result)

+---------------+---------------+
| Lluvia        |   phi(Lluvia) |
+===============+===============+
| Lluvia(False) |        0.2921 |
+---------------+---------------+
| Lluvia(True)  |        0.7079 |
+---------------+---------------+
